In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_csv('../data/urbanthreads_q4.csv')

In [4]:
df.head(3)

,Order ID,Order Date,Customer ID,Type,SKU,Category,Product Name,Unit Price,Qty,Total,...,Source,City,State,Warehouse,Del Date,Status,Return?,Reason,Ret Rec,Refund
0,UT-100001,2024-10-01,CUST_10001,New,TOP-CAS-M,Tops,Classic Cotton Tee,1299,2,2598,...,Website,Mumbai,Maharashtra,Mumbai,2024-10-05,Delivered,No,NaN,NaN,Not Applicable
1,UT-100002,2024-10-01,CUST_10002,Returning,DRS-FRM-S,Dresses,Floral Midi Dress,2499,1,2499,...,Mobile App,Delhi,Delhi,Delhi,2024-10-04,Delivered,Yes,Size Issue,2024-10-15,Processed
2,UT-100003,2024-01-10,CUST_10003,new,BOT-JNS-L,Bottoms,Slim Fit Jeans,1899,1,1899,...,Website,Bangalore,Karnataka,Bangalore,2024-10-06,Delivered,No,NaN,NaN,Not Applicable


In [6]:
df.shape

(346, 23)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order ID       346 non-null    object
 1   Order Date     346 non-null    object
 2   Customer ID    346 non-null    object
 3   Type           346 non-null    object
 4   SKU            346 non-null    object
 5   Category       346 non-null    object
 6   Product Name   346 non-null    object
 7   Unit Price     346 non-null    int64 
 8   Qty            346 non-null    int64 
 9   Total          346 non-null    int64 
 10  Discount Code  140 non-null    object
 11  Disc %         346 non-null    int64 
 12  Payment        346 non-null    object
 13  Source         346 non-null    object
 14  City           346 non-null    object
 15  State          346 non-null    object
 16  Warehouse      346 non-null    object
 17  Del Date       328 non-null    object
 18  Status         346 non-null   

In [11]:
df.columns.tolist()

['Order ID',
 'Order Date',
 'Customer ID',
 'Type',
 'SKU',
 'Category',
 'Product Name',
 'Unit Price',
 'Qty',
 'Total',
 'Discount Code',
 'Disc %',
 'Payment',
 'Source',
 'City',
 'State',
 'Warehouse',
 'Del Date',
 'Status',
 'Return?',
 'Reason',
 'Ret Rec',
 'Refund']

In [12]:
cleaning_log = []

In [16]:
cleaning_log.append({
    "issue": " missing return_reason when return_initiated = No",
    "type": "expected",
    "action": "left as blank",
    
}
)

In [17]:
pd.DataFrame(cleaning_log)

,issue,type,action
0,missing return_reason wheen return_initiated ...,expected,left as blank
1,missing return_reason when return_initiated = No,expected,left as blank


In [18]:
df_raw=df.copy()

In [19]:
df_raw.head(3)

,Order ID,Order Date,Customer ID,Type,SKU,Category,Product Name,Unit Price,Qty,Total,...,Source,City,State,Warehouse,Del Date,Status,Return?,Reason,Ret Rec,Refund
0,UT-100001,2024-10-01,CUST_10001,New,TOP-CAS-M,Tops,Classic Cotton Tee,1299,2,2598,...,Website,Mumbai,Maharashtra,Mumbai,2024-10-05,Delivered,No,NaN,NaN,Not Applicable
1,UT-100002,2024-10-01,CUST_10002,Returning,DRS-FRM-S,Dresses,Floral Midi Dress,2499,1,2499,...,Mobile App,Delhi,Delhi,Delhi,2024-10-04,Delivered,Yes,Size Issue,2024-10-15,Processed
2,UT-100003,2024-01-10,CUST_10003,new,BOT-JNS-L,Bottoms,Slim Fit Jeans,1899,1,1899,...,Website,Bangalore,Karnataka,Bangalore,2024-10-06,Delivered,No,NaN,NaN,Not Applicable
